# Portfolio Optimization - Cleaned Version
## Essential cells only for sector rotation strategy analysis

## Cell 1.0: Import Libraries and Setup

In [ ]:
# Import necessary libraries
import yfinance as yf
from fredapi import Fred
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Initialize FRED API (replace with your API key)
FRED_API_KEY = 'your_fred_api_key_here'
fred = Fred(api_key=FRED_API_KEY)

print('Libraries imported successfully!')

## Cell 2.0: Fetch Sector ETF Data

In [ ]:
# Define sector ETFs and date range
sector_etfs = [
    'XLK',  # Technology
    'XLV',  # Healthcare
    'XLF',  # Financials
    'XLY',  # Consumer Discretionary
    'XLC',  # Communication Services
    'XLI',  # Industrials
    'XLP',  # Consumer Staples
    'XLE',  # Energy
    'XLB',  # Materials
    'XLRE', # Real Estate
    'XLU',  # Utilities
    'SPY'   # S&P 500 benchmark
]

start_date = '2010-01-01'
end_date = '2025-12-30'

# Fetch adjusted close prices
print('Fetching sector ETF data...')
sector_data = yf.download(sector_etfs, start=start_date, end=end_date)['Adj Close']
sector_data = sector_data.ffill().bfill()

print(f'Data fetched for {len(sector_etfs)} ETFs from {start_date} to {end_date}')
print(f'Shape: {sector_data.shape}')
sector_data.head()

## Cell 3.0: Fetch Macroeconomic Data

In [ ]:
# Fetch macroeconomic indicators from FRED
print('Fetching macroeconomic data from FRED...')

# GDP Growth Rate
gdp = fred.get_series('GDP', observation_start=start_date, observation_end=end_date)
gdp_growth = gdp.pct_change(periods=4) * 100  # Year-over-year growth

# Consumer Price Index (CPI) - Inflation
cpi = fred.get_series('CPIAUCSL', observation_start=start_date, observation_end=end_date)
inflation = cpi.pct_change(periods=12) * 100  # Year-over-year inflation

# Unemployment Rate
unemployment = fred.get_series('UNRATE', observation_start=start_date, observation_end=end_date)

# Federal Funds Rate
fed_funds = fred.get_series('FEDFUNDS', observation_start=start_date, observation_end=end_date)

# Combine into a single DataFrame
macro_data = pd.DataFrame({
    'GDP_Growth': gdp_growth,
    'Inflation': inflation,
    'Unemployment': unemployment,
    'Fed_Funds_Rate': fed_funds
})

# Resample to daily frequency and forward fill
macro_data = macro_data.resample('D').ffill()

print('Macroeconomic data fetched successfully!')
print(f'Shape: {macro_data.shape}')
macro_data.head()

## Cell 4.0: Calculate Returns and Relative Performance

In [ ]:
# Calculate daily returns
daily_returns = sector_data.pct_change().dropna()

# Calculate cumulative returns
cumulative_returns = (1 + daily_returns).cumprod()

# Calculate rolling 20-day (1 month) returns
rolling_20d_returns = sector_data.pct_change(periods=20)

# Calculate rolling 60-day (3 month) returns
rolling_60d_returns = sector_data.pct_change(periods=60)

# Calculate relative performance vs SPY
sector_etfs_only = [etf for etf in sector_etfs if etf != 'SPY']
relative_performance = pd.DataFrame()
for etf in sector_etfs_only:
    relative_performance[etf] = rolling_60d_returns[etf] - rolling_60d_returns['SPY']

print('Returns and relative performance calculated!')
print(f'Daily returns shape: {daily_returns.shape}')
print(f'Relative performance shape: {relative_performance.shape}')
relative_performance.head()

## Cell 5.0: Generate Trading Signals

In [ ]:
# Define signal generation function
def generate_signals(relative_perf, threshold=0.05):
    """
    Generate trading signals based on relative performance.
    Buy top performing sectors, sell underperforming ones.
    
    Parameters:
    - relative_perf: DataFrame of relative performance vs benchmark
    - threshold: Threshold for significant outperformance/underperformance
    
    Returns:
    - signals: DataFrame with trading signals (1=buy, -1=sell, 0=hold)
    """
    signals = pd.DataFrame(index=relative_perf.index, columns=relative_perf.columns)
    
    for date in relative_perf.index:
        perf = relative_perf.loc[date]
        if perf.isna().all():
            signals.loc[date] = 0
            continue
        
        # Buy signal: outperformance above threshold
        signals.loc[date] = np.where(perf > threshold, 1, 0)
        
        # Sell signal: underperformance below negative threshold
        signals.loc[date] = np.where(perf < -threshold, -1, signals.loc[date])
    
    return signals.fillna(0)

# Generate signals with fixed threshold
fixed_threshold = 0.05  # 5% threshold
signals_fixed = generate_signals(relative_performance, threshold=fixed_threshold)

print(f'Trading signals generated with fixed threshold of {fixed_threshold*100}%')
print(f'Signals shape: {signals_fixed.shape}')
signals_fixed.head()

## Cell 5.1: Generate Dynamic Threshold Signals

In [ ]:
# Generate signals with dynamic threshold based on volatility
def generate_dynamic_signals(relative_perf, base_threshold=0.05, lookback=60):
    """
    Generate trading signals with dynamic threshold based on market volatility.
    
    Parameters:
    - relative_perf: DataFrame of relative performance vs benchmark
    - base_threshold: Base threshold multiplier
    - lookback: Rolling window for volatility calculation
    
    Returns:
    - signals: DataFrame with trading signals
    """
    signals = pd.DataFrame(index=relative_perf.index, columns=relative_perf.columns)
    
    # Calculate rolling volatility
    rolling_vol = relative_perf.rolling(window=lookback).std()
    
    for date in relative_perf.index:
        perf = relative_perf.loc[date]
        vol = rolling_vol.loc[date]
        
        if perf.isna().all() or vol.isna().all():
            signals.loc[date] = 0
            continue
        
        # Dynamic threshold adjusts with volatility
        dynamic_threshold = base_threshold * (1 + vol)
        
        # Buy signal: outperformance above dynamic threshold
        signals.loc[date] = np.where(perf > dynamic_threshold, 1, 0)
        
        # Sell signal: underperformance below negative dynamic threshold
        signals.loc[date] = np.where(perf < -dynamic_threshold, -1, signals.loc[date])
    
    return signals.fillna(0)

signals_dynamic = generate_dynamic_signals(relative_performance)

print('Dynamic threshold signals generated!')
print(f'Signals shape: {signals_dynamic.shape}')
signals_dynamic.head()

## Cell 6.0: Backtest Fixed Strategy

In [ ]:
# Backtest function
def backtest_strategy(signals, returns, initial_capital=100000, transaction_cost=0.001):
    """
    Backtest a trading strategy with transaction costs.
    
    Parameters:
    - signals: DataFrame with trading signals
    - returns: DataFrame with daily returns
    - initial_capital: Starting capital
    - transaction_cost: Transaction cost as a fraction (e.g., 0.001 = 0.1%)
    
    Returns:
    - portfolio_value: Series of portfolio values over time
    - positions: DataFrame of positions held
    - trades: Number of trades executed
    """
    # Align signals and returns
    common_dates = signals.index.intersection(returns.index)
    signals = signals.loc[common_dates]
    returns = returns.loc[common_dates]
    
    # Initialize portfolio
    portfolio_value = pd.Series(index=common_dates, dtype=float)
    portfolio_value.iloc[0] = initial_capital
    
    positions = pd.DataFrame(0, index=common_dates, columns=signals.columns)
    trades = 0
    
    # Equal weight allocation for buy signals
    for i in range(len(common_dates)):
        date = common_dates[i]
        
        if i == 0:
            # Initial positions based on signals
            buy_signals = signals.loc[date] == 1
            num_positions = buy_signals.sum()
            if num_positions > 0:
                positions.loc[date] = buy_signals.astype(int) / num_positions
        else:
            prev_date = common_dates[i-1]
            
            # Calculate portfolio return
            daily_return = (positions.loc[prev_date] * returns.loc[date]).sum()
            portfolio_value.iloc[i] = portfolio_value.iloc[i-1] * (1 + daily_return)
            
            # Update positions if signals change
            buy_signals = signals.loc[date] == 1
            num_positions = buy_signals.sum()
            
            if num_positions > 0:
                new_positions = buy_signals.astype(int) / num_positions
            else:
                new_positions = positions.loc[prev_date] * 0
            
            # Count trades (position changes)
            if not new_positions.equals(positions.loc[prev_date]):
                trades += (new_positions != positions.loc[prev_date]).sum()
                # Apply transaction costs
                cost = portfolio_value.iloc[i] * transaction_cost * (new_positions != positions.loc[prev_date]).sum()
                portfolio_value.iloc[i] -= cost
            
            positions.loc[date] = new_positions
    
    return portfolio_value, positions, trades

# Backtest fixed threshold strategy
print('Backtesting fixed threshold strategy...')
portfolio_fixed, positions_fixed, trades_fixed = backtest_strategy(
    signals_fixed, 
    daily_returns[sector_etfs_only],
    initial_capital=100000,
    transaction_cost=0.001
)

print(f'Fixed strategy completed!')
print(f'Total trades: {trades_fixed}')
print(f'Final portfolio value: ${portfolio_fixed.iloc[-1]:,.2f}')

## Cell 7.0: Backtest Dynamic Strategy

In [ ]:
# Backtest dynamic threshold strategy
print('Backtesting dynamic threshold strategy...')
portfolio_dynamic, positions_dynamic, trades_dynamic = backtest_strategy(
    signals_dynamic,
    daily_returns[sector_etfs_only],
    initial_capital=100000,
    transaction_cost=0.001
)

print(f'Dynamic strategy completed!')
print(f'Total trades: {trades_dynamic}')
print(f'Final portfolio value: ${portfolio_dynamic.iloc[-1]:,.2f}')

## Cell 7.1: Backtest Dynamic Strategy Without Transaction Costs

In [ ]:
# Backtest dynamic threshold strategy without transaction costs
print('Backtesting dynamic threshold strategy (no transaction costs)...')
portfolio_dynamic_nocost, positions_dynamic_nocost, trades_dynamic_nocost = backtest_strategy(
    signals_dynamic,
    daily_returns[sector_etfs_only],
    initial_capital=100000,
    transaction_cost=0.0
)

print(f'Dynamic strategy (no costs) completed!')
print(f'Total trades: {trades_dynamic_nocost}')
print(f'Final portfolio value: ${portfolio_dynamic_nocost.iloc[-1]:,.2f}')

## Cell 8.0: Backtest Monthly Strategy

In [ ]:
# Monthly rebalancing strategy
def backtest_monthly_strategy(relative_perf, returns, initial_capital=100000, transaction_cost=0.001):
    """
    Backtest a monthly rebalancing strategy that invests in top 3 sectors.
    
    Parameters:
    - relative_perf: DataFrame of relative performance vs benchmark
    - returns: DataFrame with daily returns
    - initial_capital: Starting capital
    - transaction_cost: Transaction cost as a fraction
    
    Returns:
    - portfolio_value: Series of portfolio values over time
    """
    # Get month-end dates
    monthly_dates = relative_perf.resample('M').last().index
    
    # Align with available data
    common_dates = returns.index
    portfolio_value = pd.Series(index=common_dates, dtype=float)
    portfolio_value.iloc[0] = initial_capital
    
    positions = pd.DataFrame(0, index=common_dates, columns=returns.columns)
    current_positions = pd.Series(0, index=returns.columns)
    
    rebalance_dates = [d for d in monthly_dates if d in common_dates]
    
    for i in range(len(common_dates)):
        date = common_dates[i]
        
        # Check if it's a rebalancing date
        if date in rebalance_dates and date in relative_perf.index:
            # Select top 3 sectors based on relative performance
            perf = relative_perf.loc[date].dropna()
            if len(perf) >= 3:
                top_3 = perf.nlargest(3).index
                new_positions = pd.Series(0, index=returns.columns)
                new_positions[top_3] = 1/3  # Equal weight
                
                current_positions = new_positions
        
        positions.loc[date] = current_positions
        
        # Calculate portfolio return
        if i > 0:
            prev_date = common_dates[i-1]
            daily_return = (positions.loc[date] * returns.loc[date]).sum()
            portfolio_value.iloc[i] = portfolio_value.iloc[i-1] * (1 + daily_return)
            
            # Apply transaction costs if positions changed
            if not positions.loc[date].equals(positions.loc[prev_date]):
                trades = (positions.loc[date] != positions.loc[prev_date]).sum()
                cost = portfolio_value.iloc[i] * transaction_cost * trades
                portfolio_value.iloc[i] -= cost
    
    return portfolio_value

# Backtest monthly rebalancing strategy
print('Backtesting monthly rebalancing strategy...')
portfolio_monthly = backtest_monthly_strategy(
    relative_performance,
    daily_returns[sector_etfs_only],
    initial_capital=100000,
    transaction_cost=0.001
)

print(f'Monthly strategy completed!')
print(f'Final portfolio value: ${portfolio_monthly.iloc[-1]:,.2f}')

## Cell 9.0: Calculate Performance Metrics

In [ ]:
# Calculate performance metrics
def calculate_metrics(portfolio_value, initial_capital=100000):
    """
    Calculate key performance metrics for a portfolio.
    
    Parameters:
    - portfolio_value: Series of portfolio values
    - initial_capital: Initial investment amount
    
    Returns:
    - metrics: Dictionary of performance metrics
    """
    # Total return
    total_return = (portfolio_value.iloc[-1] / initial_capital - 1) * 100
    
    # Annualized return
    years = len(portfolio_value) / 252  # Assuming 252 trading days per year
    annualized_return = ((portfolio_value.iloc[-1] / initial_capital) ** (1/years) - 1) * 100
    
    # Daily returns
    daily_rets = portfolio_value.pct_change().dropna()
    
    # Annualized volatility
    volatility = daily_rets.std() * np.sqrt(252) * 100
    
    # Sharpe ratio (assuming 2% risk-free rate)
    risk_free_rate = 0.02
    sharpe_ratio = (annualized_return/100 - risk_free_rate) / (volatility/100)
    
    # Maximum drawdown
    cumulative = portfolio_value / portfolio_value.cummax()
    max_drawdown = (cumulative.min() - 1) * 100
    
    return {
        'Total Return (%)': total_return,
        'Annualized Return (%)': annualized_return,
        'Volatility (%)': volatility,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown (%)': max_drawdown
    }

# Calculate metrics for all strategies
metrics_fixed = calculate_metrics(portfolio_fixed)
metrics_dynamic = calculate_metrics(portfolio_dynamic)
metrics_dynamic_nocost = calculate_metrics(portfolio_dynamic_nocost)
metrics_monthly = calculate_metrics(portfolio_monthly)

# Calculate SPY buy-and-hold metrics
spy_value = (1 + daily_returns['SPY']).cumprod() * 100000
spy_value = spy_value[spy_value.index.isin(portfolio_fixed.index)]
metrics_spy = calculate_metrics(spy_value)

# Create comparison DataFrame
metrics_comparison = pd.DataFrame({
    'Fixed Threshold': metrics_fixed,
    'Dynamic Threshold': metrics_dynamic,
    'Dynamic (No Costs)': metrics_dynamic_nocost,
    'Monthly Rebalancing': metrics_monthly,
    'SPY Buy & Hold': metrics_spy
})

print('\nPerformance Metrics Comparison:')
print('='*80)
print(metrics_comparison.round(2))

## Cell 10.0: Visualize Results

In [ ]:
# Visualize cumulative returns
fig, ax = plt.subplots(figsize=(14, 8))

# Normalize all portfolios to start at 100
portfolio_fixed_norm = (portfolio_fixed / portfolio_fixed.iloc[0]) * 100
portfolio_dynamic_norm = (portfolio_dynamic / portfolio_dynamic.iloc[0]) * 100
portfolio_dynamic_nocost_norm = (portfolio_dynamic_nocost / portfolio_dynamic_nocost.iloc[0]) * 100
portfolio_monthly_norm = (portfolio_monthly / portfolio_monthly.iloc[0]) * 100
spy_norm = (spy_value / spy_value.iloc[0]) * 100

# Plot all strategies
ax.plot(portfolio_fixed_norm.index, portfolio_fixed_norm, label='Fixed Threshold', linewidth=2)
ax.plot(portfolio_dynamic_norm.index, portfolio_dynamic_norm, label='Dynamic Threshold', linewidth=2)
ax.plot(portfolio_dynamic_nocost_norm.index, portfolio_dynamic_nocost_norm, 
        label='Dynamic (No Costs)', linewidth=2, linestyle='--')
ax.plot(portfolio_monthly_norm.index, portfolio_monthly_norm, label='Monthly Rebalancing', linewidth=2)
ax.plot(spy_norm.index, spy_norm, label='SPY Buy & Hold', linewidth=2, color='black', alpha=0.7)

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Portfolio Value (Normalized to 100)', fontsize=12)
ax.set_title('Sector Rotation Strategies - Cumulative Returns Comparison', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('Cumulative returns visualization complete!')

## Cell 11.0: Calculate Trade Frequency

In [ ]:
# Analyze trade frequency
def analyze_trade_frequency(positions):
    """
    Analyze trading frequency and turnover.
    
    Parameters:
    - positions: DataFrame of positions over time
    
    Returns:
    - analysis: Dictionary with trade frequency metrics
    """
    # Count position changes
    position_changes = (positions.diff() != 0).sum().sum()
    
    # Calculate trading days
    trading_days = len(positions)
    
    # Average trades per month
    months = trading_days / 21  # Approximate trading days per month
    trades_per_month = position_changes / months
    
    # Portfolio turnover (average absolute position change)
    turnover = positions.diff().abs().sum().sum() / len(positions)
    
    return {
        'Total Position Changes': position_changes,
        'Trading Days': trading_days,
        'Avg Trades per Month': trades_per_month,
        'Avg Daily Turnover': turnover
    }

# Analyze both strategies
trade_freq_fixed = analyze_trade_frequency(positions_fixed)
trade_freq_dynamic = analyze_trade_frequency(positions_dynamic)

# Create comparison DataFrame
trade_frequency_comparison = pd.DataFrame({
    'Fixed Threshold': trade_freq_fixed,
    'Dynamic Threshold': trade_freq_dynamic
})

print('\nTrade Frequency Analysis:')
print('='*60)
print(trade_frequency_comparison.round(2))

# Visualize monthly trade counts
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Fixed strategy
monthly_changes_fixed = (positions_fixed.diff() != 0).resample('M').sum().sum(axis=1)
axes[0].bar(range(len(monthly_changes_fixed)), monthly_changes_fixed.values, alpha=0.7)
axes[0].set_title('Fixed Threshold - Monthly Trade Count', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Number of Trades')
axes[0].grid(True, alpha=0.3)

# Dynamic strategy
monthly_changes_dynamic = (positions_dynamic.diff() != 0).resample('M').sum().sum(axis=1)
axes[1].bar(range(len(monthly_changes_dynamic)), monthly_changes_dynamic.values, alpha=0.7, color='orange')
axes[1].set_title('Dynamic Threshold - Monthly Trade Count', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Number of Trades')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('\nTrade frequency analysis complete!')